<a href="https://www.kaggle.com/nur988/traffic-congestion-xgboost?scriptVersionId=90376908" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn.functional as F
from tqdm.notebook import tqdm
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold,cross_val_score,GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df=pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
train_df.head(20)

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
5,5,1991-04-01 00:00:00,0,1,SB,58
6,6,1991-04-01 00:00:00,0,1,WB,26
7,7,1991-04-01 00:00:00,0,2,EB,31
8,8,1991-04-01 00:00:00,0,2,NB,49
9,9,1991-04-01 00:00:00,0,2,SB,46


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848835 entries, 0 to 848834
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   row_id      848835 non-null  int64 
 1   time        848835 non-null  object
 2   x           848835 non-null  int64 
 3   y           848835 non-null  int64 
 4   direction   848835 non-null  object
 5   congestion  848835 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 38.9+ MB


In [4]:
train_df.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60


In [5]:
def encode(x):
    if x=='EB':
        return 1
    elif x=='WB':
        return 2
    elif x=='SB':
        return 3
    else:
        return 4

In [6]:
def preprocess(data):
    data['time'] = pd.to_datetime(data['time'])
    data['month'] = data['time'].dt.month
    data['weekday'] = data['time'].dt.weekday
    data['hour'] = data['time'].dt.hour
    data['minute'] = data['time'].dt.minute
    data['is_month_start'] = data['time'].dt.is_month_start.astype('int')
    data['is_month_end'] = data['time'].dt.is_month_end.astype('int')
    data['is_weekend'] = (data['time'].dt.dayofweek > 5).astype('int')
    data['is_afternoon'] = (data['time'].dt.hour > 12).astype('int')
    data['road'] = data['x'].astype(str) + data['y'].astype(str) + data['direction']
    cat_feat=['road']
    le=LabelEncoder()
    le.fit(data[cat_feat])
    data['road']=le.transform(data['road'])
    
    data['moment']  = data['time'].dt.hour * 3 + data['time'].dt.minute // 20 
    
    data = data.drop(['row_id', 'direction'], axis=1)
    
    return data
    

In [7]:
train_df=preprocess(train_df)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
train_df.head()

,time,x,y,congestion,month,weekday,hour,minute,is_month_start,is_month_end,is_weekend,is_afternoon,road,moment
0,1991-04-01,0,0,70,4,0,0,0,1,0,0,0,0,0
1,1991-04-01,0,0,49,4,0,0,0,1,0,0,0,1,0
2,1991-04-01,0,0,24,4,0,0,0,1,0,0,0,2,0
3,1991-04-01,0,1,18,4,0,0,0,1,0,0,0,3,0
4,1991-04-01,0,1,60,4,0,0,0,1,0,0,0,4,0


In [9]:
X=train_df.drop(['congestion','time'],axis=1)
Y=train_df.congestion.values

In [10]:
X_train,X_valid,Y_train,Y_valid=train_test_split(X,Y,test_size=0.3)

In [11]:
X_train.head()

,x,y,month,weekday,hour,minute,is_month_start,is_month_end,is_weekend,is_afternoon,road,moment
598786,0,1,8,2,6,20,0,0,0,0,6,19
631283,0,1,8,2,5,40,0,0,0,0,3,17
27195,1,1,4,5,19,20,0,0,0,1,25,58
658256,0,0,8,1,1,20,0,0,0,0,1,4
4157,2,3,4,0,21,0,1,0,0,1,62,63


In [12]:
X.head()

,x,y,month,weekday,hour,minute,is_month_start,is_month_end,is_weekend,is_afternoon,road,moment
0,0,0,4,0,0,0,1,0,0,0,0,0
1,0,0,4,0,0,0,1,0,0,0,1,0
2,0,0,4,0,0,0,1,0,0,0,2,0
3,0,1,4,0,0,0,1,0,0,0,3,0
4,0,1,4,0,0,0,1,0,0,0,4,0


In [13]:
model = XGBRegressor(n_estimators=400, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [14]:
# cv=RepeatedKFold(n_splits=5,n_repeats=3,random_state=1)
# scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# scores=absolute(scores)

In [15]:
def mae(x,y):
    val=abs(x-y)
    return val.mean()

In [16]:
model.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             eta=0.1, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.100000001,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=400, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
pred=model.predict(X_valid)
print(mae(pred,Y_valid))

6.292079280831086


In [18]:
test_df=pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')

In [19]:
test_df.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [20]:
test_df=preprocess(test_df)
test_df.head()

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,time,x,y,month,weekday,hour,minute,is_month_start,is_month_end,is_weekend,is_afternoon,road,moment
0,1991-09-30 12:00:00,0,0,9,0,12,0,0,1,0,0,0,36
1,1991-09-30 12:00:00,0,0,9,0,12,0,0,1,0,0,1,36
2,1991-09-30 12:00:00,0,0,9,0,12,0,0,1,0,0,2,36
3,1991-09-30 12:00:00,0,1,9,0,12,0,0,1,0,0,3,36
4,1991-09-30 12:00:00,0,1,9,0,12,0,0,1,0,0,4,36


In [21]:
test_df=test_df.drop(['time'],axis=1)

In [22]:
prediction=model.predict(test_df)

In [23]:
submission=pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
submission['congestion']=prediction
submission.to_csv('submission.csv',index=False)
submission.head()

,row_id,congestion
0,848835,46.901211
1,848836,31.356274
2,848837,44.780827
3,848838,24.995144
4,848839,68.140381
